In [1]:
import csv
from scipy import ndimage
from scipy import linalg
import numpy as np
import nibabel as nib
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from operator import itemgetter
import pickle

In [2]:
import random

index = np.array(range(119))
random.seed(40)
random.shuffle(index)

train_index = index[0:84]
val_index = index[84:96]
test_index = index[96:119]

#train_index.astype(np.int64)
#val_index.astype(np.int64)
#test_index.astype(np.int64)

In [13]:
dataimage_train = [dataimage[j] for j in train_index]
print(dataimage_train)

NameError: name 'dataimage' is not defined

In [4]:
from torch.utils.data import Dataset, DataLoader

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

class ADNI_Dataset(Dataset):
    
    def __init__(self, ptype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        if(ptype == 'train'):
            with open("dataimage_train.txt", "rb") as fp:   # Unpickling
                b_train = pickle.load(fp)
            with open("datacsv_train.txt", "rb") as fp:   # Unpickling
                c_train = pickle.load(fp)
            self.csv = c_train
            self.image = b_train
        
        if(ptype == 'val'):
            with open("dataimage_val.txt", "rb") as fp:   # Unpickling
                b_val = pickle.load(fp)
            with open("datacsv_val.txt", "rb") as fp:   # Unpickling
                c_val = pickle.load(fp)
            self.csv = c_val
            self.image = b_val
        
        if(ptype == 'test'):
            with open("dataimage_test.txt", "rb") as fp:   # Unpickling
                b_test = pickle.load(fp)
            with open("datacsv_test.txt", "rb") as fp:   # Unpickling
                c_test = pickle.load(fp)
            self.csv = c_test
            self.image = b_test

        print("great job dude")
        #c[0][0] and b[0][0]'s patient name formats don't conform

        

    def __len__(self):#return int: number of mri
        return len(self.image)

    def __getitem__(self, idx):#input the dataset and desires i-th item
        img_name = self.csv[idx][0]
        image = self.image[idx][1]
        group = self.csv[idx][1]
        sample = {'image': image, 'patient_id': img_name, 'groups': group} 

        return sample


In [5]:
ADNI_Data_train = ADNI_Dataset('train')

ADNI_Data_val = ADNI_Dataset('val')

ADNI_Data_test = ADNI_Dataset('test')



dataloader_train = DataLoader(ADNI_Data_train, batch_size=1,
                        shuffle=True, num_workers=4)
dataloader_val = DataLoader(ADNI_Data_val, batch_size=1,
                        shuffle=True, num_workers=4)
dataloader_test = DataLoader(ADNI_Data_test, batch_size=1,
                        shuffle=True, num_workers=4)

print("finished")

great job dude
great job dude
great job dude
finished


In [6]:
import torch.nn as nn
import torch.nn.functional as F

#initial: 256*256*160 * 1(feature map number)
#128*128*80 * 2
#64*64*40 * 4
#32*32*20 * 8
#16*16*10 * 16
#8*8*5 * 32
#Linear: 32 * 8 * 8 * 5 
#10000--->1024 --->256 ---> 64 --->3  
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 16, 5, padding=2)
        self.conv5 = nn.Conv3d(16, 32, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(32 * 8 * 8 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 32 * 8 * 8 * 5)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

net = Net().cuda()

print("finished")

finished


In [97]:
import torch.nn as nn
import torch.nn.functional as F
class Light_Net(nn.Module):
    def __init__(self):
        super(Light_Net, self).__init__()
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 2, kernel_size = 5, padding=2)
        self.conv2 = nn.Conv3d(2, 4, 5, padding=2)
        self.conv3 = nn.Conv3d(4, 8, 5, padding=2)
        self.conv4 = nn.Conv3d(8, 8, 5, padding=2)
        self.conv5 = nn.Conv3d(8, 8, 5, padding=2)
        self.pool = nn.MaxPool3d(2)
        self.fc1 = nn.Linear(8 * 8 * 8 * 5, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)
        self.fc4 = nn.Linear(64, 3)

    def forward(self, x): #batch_size * 256*256*160*1
        x = self.pool(F.relu(self.conv1(x)))#batch_size * 128 * 128 * 80 * 2
        x = self.pool(F.relu(self.conv2(x)))#batch_size * 64 * 64 * 40 * 4
        x = self.pool(F.relu(self.conv3(x)))#batch_size * 32 * 32 * 20 * 8
        x = self.pool(F.relu(self.conv4(x)))#batch_size * 16 * 16 * 10 * 16
        x = self.pool(F.relu(self.conv5(x)))#batch_size * 8 * 8 * 5 * 32
        x = x.view(-1, 8 * 8 * 8 * 5)#batch_size * 10240(8*8*5*32)
        x = F.relu(self.fc1(x))#batch_size * 1024
        x = F.relu(self.fc2(x))#batch_size * 256
        x = F.relu(self.fc3(x))#batch_size * 64
        x = self.fc4(x)#batch_size * 3
        return x

#net = Light_Net().cuda()
print("finished")

finished


In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print("finished")

finished


In [9]:
import numpy as np
import torch

def ToTensor(np_array):
    tensor = torch.from_numpy(np_array)
    tensor = tensor.float()
    return tensor

epoch = 50
min_total_loss = 1000000


for i in range(epoch):
    for i_batch, sample_batched in enumerate(dataloader_train):
        print(i_batch,
              sample_batched['patient_id'])

        running_loss = 0.0
        # get the inputs
        labels_arr = np.asarray(sample_batched['groups'])
        labels_arr = labels_arr.astype(np.int)
        labels = ToTensor(labels_arr).cuda()


        images_arr = np.asarray(sample_batched['image'])
        images_arr = images_arr.astype(np.float)
        images_arr = np.expand_dims(images_arr, 1)
        images = ToTensor(images_arr).cuda()



        # zero the parameter gradients
        optimizer.zero_grad()
        
        #before actual training
        net.train()

        # forward + backward + optimize
        outputs = net(images)
        print(outputs)
        print(labels.long())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # print statistics
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))

    print('$$Finished Training This Epoch')
    
    #validation for the epoch
    net.eval()
    total_val_loss = 0
    print('Start a new val_epoch')
    for i_batch, sample_batched in enumerate(dataloader_val):
        print(i_batch,
              sample_batched['patient_id'])

        running_loss = 0.0
        # get the inputs
        labels_arr = np.asarray(sample_batched['groups'])
        labels_arr = labels_arr.astype(np.int)
        labels = ToTensor(labels_arr).cuda()

        images_arr = np.asarray(sample_batched['image'])
        images_arr = images_arr.astype(np.float)
        images_arr = np.expand_dims(images_arr, 1)
        images = ToTensor(images_arr).cuda()
        
        outputs = net(images)
        loss = criterion(outputs, labels.long())
        print('Validation')
        print('[%d] loss: %.3f' %
                (i_batch + 1, loss.item()))
        print('\n')
        
        total_val_loss = total_val_loss + loss.cpu().data.numpy()
        
        
    if(total_val_loss < min_total_loss):
        min_total_loss = total_val_loss
   
    #save model
    #best_model
    torch.save(net, 'best_model_2.pkl')

print("we're done training")

0 ['"018_S_0633"']
tensor([[ 0.1275,  0.0321, -0.0733]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[1] loss: 1.099
1 ['"016_S_1121"']
tensor([[ 0.1268,  0.0319, -0.0724]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[2] loss: 1.004
2 ['"027_S_0404"']
tensor([[ 0.1271,  0.0313, -0.0720]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[3] loss: 1.099
3 ['"116_S_0649"']
tensor([[ 0.1268,  0.0316, -0.0721]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[4] loss: 1.004
4 ['"012_S_1009"']
tensor([[ 0.1273,  0.0315, -0.0725]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[5] loss: 1.203
5 ['"136_S_1227"']
tensor([[ 0.1273,  0.0310, -0.0719]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[6] loss: 1.003
6 ['"023_S_0604"']
tensor([[ 0.1281,  0.0301, -0.0718]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cud

[56] loss: 1.212
56 ['"053_S_0507"']
tensor([[ 0.2089, -0.0443, -0.0750]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[57] loss: 0.928
57 ['"012_S_0689"']
tensor([[ 0.2122, -0.0477, -0.0747]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[58] loss: 1.185
58 ['"067_S_1253"']
tensor([[ 0.2148, -0.0499, -0.0749]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[59] loss: 1.187
59 ['"023_S_0916"']
tensor([[ 0.2165, -0.0511, -0.0754]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[60] loss: 1.189
60 ['"037_S_0303"']
tensor([[ 0.2177, -0.0514, -0.0762]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[61] loss: 1.214
61 ['"023_S_1262"']
tensor([[ 0.2182, -0.0520, -0.0761]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[62] loss: 1.190
62 ['"018_S_0369"']
tensor([[ 0.2182, -0.0517, -0.0764]], device='cuda:0', grad_fn=<ThAddmmBack

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch/serialization.py:241: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


0 ['"012_S_0689"']
tensor([[ 0.1958, -0.0630, -0.0424]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[1] loss: 1.199
1 ['"002_S_0954"']
tensor([[ 0.1943, -0.0640, -0.0399]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[2] loss: 0.941
2 ['"018_S_0406"']
tensor([[ 0.1938, -0.0653, -0.0380]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[3] loss: 0.942
3 ['"027_S_0403"']
tensor([[ 0.1940, -0.0667, -0.0366]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[4] loss: 1.172
4 ['"023_S_0061"']
tensor([[ 0.1937, -0.0685, -0.0346]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[5] loss: 1.170
5 ['"023_S_1262"']
tensor([[ 0.1930, -0.0703, -0.0319]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[6] loss: 1.206
6 ['"037_S_0303"']
tensor([[ 0.1919, -0.0712, -0.0298]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cud

[56] loss: 0.937
56 ['"007_S_1206"']
tensor([[ 0.2027, -0.1046, -0.0057]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[57] loss: 1.143
57 ['"136_S_1227"']
tensor([[ 0.2045, -0.1054, -0.0067]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[58] loss: 0.933
58 ['"023_S_1247"']
tensor([[ 0.2069, -0.1065, -0.0079]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[59] loss: 0.931
59 ['"136_S_0196"']
tensor([[ 0.2098, -0.1078, -0.0095]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[60] loss: 1.148
60 ['"051_S_1123"']
tensor([[ 0.2119, -0.1093, -0.0100]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[61] loss: 1.149
61 ['"005_S_0602"']
tensor([[ 0.2134, -0.1110, -0.0097]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[62] loss: 1.149
62 ['"018_S_0425"']
tensor([[ 0.2142, -0.1128, -0.0085]], device='cuda:0', grad_fn=<ThAddmmBack

...

tensor([[ 0.3224, -0.3263,  0.1023]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[28] loss: 0.844
28 ['"016_S_1121"']
tensor([[ 0.3234, -0.3266,  0.1016]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[29] loss: 0.843
29 ['"018_S_0425"']
tensor([[ 0.3249, -0.3271,  0.1006]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[30] loss: 1.066
30 ['"005_S_0553"']
tensor([[ 0.3258, -0.3279,  0.1004]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[31] loss: 1.066
31 ['"031_S_0830"']
tensor([[ 0.3261, -0.3288,  0.1010]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[32] loss: 0.841
32 ['"027_S_1081"']
tensor([[ 0.3271, -0.3299,  0.1012]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[33] loss: 1.497
33 ['"116_S_1249"']
tensor([[ 0.3274, -0.3299,  0.1009]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[

[83] loss: 0.825
83 ['"037_S_1225"']
tensor([[ 0.3466, -0.3525,  0.1048]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[84] loss: 0.825
$$Finished Training This Epoch
Start a new val_epoch
0 ['"128_S_1088"']
Validation
[1] loss: 0.825


1 ['"023_S_0625"']
Validation
[2] loss: 0.825


2 ['"136_S_0195"']
Validation
[3] loss: 0.825


3 ['"127_S_0260"']
Validation
[4] loss: 1.066


4 ['"067_S_0607"']
Validation
[5] loss: 0.825


5 ['"130_S_0505"']
Validation
[6] loss: 0.825


6 ['"023_S_0388"']
Validation
[7] loss: 0.825


7 ['"067_S_0290"']
Validation
[8] loss: 0.825


8 ['"116_S_1232"']
Validation
[9] loss: 1.066


9 ['"002_S_0413"']
Validation
[10] loss: 1.066


10 ['"018_S_0450"']
Validation
[11] loss: 0.825


11 ['"100_S_0190"']
Validation
[12] loss: 0.825


0 ['"018_S_0425"']
tensor([[ 0.3471, -0.3540,  0.1058]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[1] loss: 1.066
1 ['"023_S_0031"']
tensor([[ 0.3471, -0.3556,  0.1074]]

tensor([2], device='cuda:0')
[51] loss: 1.043
51 ['"126_S_0606"']
tensor([[ 0.3544, -0.3911,  0.1363]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[52] loss: 1.569
52 ['"007_S_1222"']
tensor([[ 0.3532, -0.3910,  0.1375]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[53] loss: 1.040
53 ['"037_S_0501"']
tensor([[ 0.3516, -0.3912,  0.1392]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[54] loss: 0.826
54 ['"127_S_0844"']
tensor([[ 0.3509, -0.3916,  0.1404]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[55] loss: 1.569
55 ['"002_S_0559"']
tensor([[ 0.3496, -0.3910,  0.1411]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[56] loss: 1.036
56 ['"128_S_1148"']
tensor([[ 0.3480, -0.3908,  0.1424]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[57] loss: 0.829
57 ['"021_S_0332"']
tensor([[ 0.3472, -0.3907,  0.1432]], device='

tensor([[ 0.3738, -0.3858,  0.1115]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[19] loss: 0.805
19 ['"012_S_0689"']
tensor([[ 0.3748, -0.3871,  0.1118]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[20] loss: 1.566
20 ['"136_S_0086"']
tensor([[ 0.3752, -0.3873,  0.1116]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[21] loss: 1.068
21 ['"023_S_0916"']
tensor([[ 0.3750, -0.3877,  0.1122]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[22] loss: 1.567
22 ['"032_S_1101"']
tensor([[ 0.3743, -0.3871,  0.1123]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[23] loss: 1.566
23 ['"082_S_1256"']
tensor([[ 0.3732, -0.3857,  0.1120]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[24] loss: 1.067
24 ['"116_S_0487"']
tensor([[ 0.3716, -0.3847,  0.1125]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[

tensor([0], device='cuda:0')
[74] loss: 0.818
74 ['"023_S_0855"']
tensor([[ 0.3538, -0.3540,  0.0990]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[75] loss: 0.819
75 ['"094_S_1293"']
tensor([[ 0.3537, -0.3551,  0.1002]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[76] loss: 0.819
76 ['"051_S_1123"']
tensor([[ 0.3542, -0.3563,  0.1009]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[77] loss: 1.072
77 ['"127_S_0622"']
tensor([[ 0.3542, -0.3576,  0.1023]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([2], device='cuda:0')
[78] loss: 1.071
78 ['"027_S_1082"']
tensor([[ 0.3536, -0.3590,  0.1043]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([1], device='cuda:0')
[79] loss: 1.532
79 ['"130_S_0449"']
tensor([[ 0.3526, -0.3594,  0.1057]], device='cuda:0', grad_fn=<ThAddmmBackward>)
tensor([0], device='cuda:0')
[80] loss: 0.821
80 ['"023_S_0061"']
tensor([[ 0.3524, -0.3600,  0.1065]], device='

In [15]:
import torch
#test after finish all epochs

#load the best_model back 
final_net = torch.load('best_model_2.pkl')
final_net.eval()
correct_sum = 0.
total_sum = 0.
for i_batch, sample_batched in enumerate(dataloader_test):
    print(i_batch,
          sample_batched['patient_id'])

    running_loss = 0.0
    # get the inputs
    labels_arr = np.asarray(sample_batched['groups'])
    labels_arr = labels_arr.astype(np.int)
    labels = ToTensor(labels_arr).cuda()

    images_arr = np.asarray(sample_batched['image'])
    images_arr = images_arr.astype(np.float)
    images_arr = np.expand_dims(images_arr, 1)
    images = ToTensor(images_arr).cuda()

    outputs = final_net(images)#n * 3
    results_type = np.argmax(outputs.cpu().data.numpy(), axis=1)
    results_type = ToTensor(results_type)
    print(outputs)

    #calculate accuracy
    labels = labels.cpu()
    print("\n", results_type, " ", labels)

    
    correct_sum += (labels == results_type)
    total_sum += 1


print("correct_sum_final", correct_sum)
print(total_sum)


0 ['"136_S_0300"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([1.])
1 ['"016_S_1117"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([0.])
2 ['"002_S_1018"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([1.])
3 ['"052_S_1250"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([2.])
4 ['"136_S_0426"']
tensor([[ 0.3504, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([1.])
5 ['"005_S_0572"']
tensor([[ 0.3504, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([0.])
6 ['"023_S_0926"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<ThAddmmBackward>)

 tensor([0.])   tensor([2.])
7 ['"023_S_0139"']
tensor([[ 0.3503, -0.3610,  0.1096]], device='cuda:0', grad_fn=<